# 메시지(Messages)

메시지는 LangChain에서 모델의 컨텍스트를 나타내는 기본 단위입니다. 모델의 입력과 출력을 나타내며, LLM과 상호 작용할 때 대화 상태를 나타내는 데 필요한 콘텐츠와 메타데이터를 모두 포함합니다.

메시지는 다음을 포함하는 객체입니다:

- **역할(Role)** - 메시지 유형을 식별 (예: `system`, `user`, `assistant`)
- **콘텐츠(Content)** - 메시지의 실제 콘텐츠를 나타냄 (텍스트, 이미지, 오디오, 문서 등)
- **메타데이터(Metadata)** - 응답 정보, 메시지 ID, 토큰 사용량 등의 선택적 필드

LangChain은 모델 제공자에 관계없이 일관된 동작을 보장하는 표준 메시지 유형을 제공합니다.

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("LangChain-Advanced-Tutorial")

## 기본 사용법

메시지를 사용하는 가장 간단한 방법은 메시지 객체를 생성하고 모델을 호출할 때 전달하는 것입니다.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 모델 초기화
model = init_chat_model("openai:gpt-4.1-mini")

# 메시지 객체 생성
system_msg = SystemMessage("당신은 친절한 Assistant입니다.")
human_msg = HumanMessage("안녕하세요. 반갑습니다.")

# 채팅 모델과 함께 사용
messages = [system_msg, human_msg]
response = model.invoke(messages)  # AIMessage 반환
print(response)

### 텍스트 프롬프트

텍스트 프롬프트는 문자열입니다. 대화 기록을 유지할 필요가 없는 간단한 생성 작업에 이상적입니다.

**텍스트 프롬프트를 사용하는 경우**

- 단일의 독립적인 요청이 있는 경우
- 대화 기록이 필요하지 않은 경우
- 최소한의 코드 복잡성을 원하는 경우

In [ ]:
# 단일 문자열로 간단한 요청
response = model.invoke("대한민국의 수도는?")
print(response.content)

### 메시지 프롬프트

메시지 객체 목록을 제공하여 모델에 메시지 목록을 전달할 수 있습니다.

**메시지 프롬프트를 사용하는 경우**

- 다중 턴 대화를 관리하는 경우
- 멀티모달 콘텐츠(이미지, 오디오, 파일)를 작업하는 경우
- 시스템 지침을 포함하는 경우

In [ ]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage

# 메시지 객체를 사용한 대화
messages = [
    SystemMessage("당신은 친절한 어시스턴트입니다."),
    HumanMessage("대한민국의 수도는?"),
    AIMessage("대한민국의 수도는 서울입니다."),
    HumanMessage("영어로 작성해줘."),
]

response = model.invoke(messages)
print(response.content)

### 딕셔너리 형식

OpenAI 채팅 완성 형식으로 메시지를 직접 지정할 수도 있습니다.

In [ ]:
# 딕셔너리 형식으로 메시지 지정
messages = [
    {"role": "system", "content": "당신은 친절한 어시스턴트입니다."},
    {"role": "user", "content": "대한민국의 수도는?"},
    {"role": "assistant", "content": "대한민국의 수도는 서울입니다."},
    {"role": "user", "content": "영어로 작성해줘."},
]

response = model.invoke(messages)
print(response.content)

## 메시지 유형

LangChain은 다음과 같은 메시지 유형을 제공합니다.

- **시스템 메시지(System Message)** - 모델의 동작 방식을 알려주고 상호 작용을 위한 컨텍스트를 제공
- **사용자 메시지(Human Message)** - 사용자 입력 및 모델과의 상호 작용을 나타냄
- **AI 메시지(AI Message)** - 텍스트 콘텐츠, 도구 호출 및 메타데이터를 포함한 모델이 생성한 응답
- **도구 메시지(Tool Message)** - 도구 호출의 출력을 나타냄

### 시스템 메시지

시스템 메시지는 모델의 동작을 준비하는 초기 지침 세트를 나타냅니다. 시스템 메시지를 사용하여 톤을 설정하고, 모델의 역할을 정의하며, 응답에 대한 가이드라인을 수립할 수 있습니다.

In [ ]:
from langchain.messages import SystemMessage, HumanMessage

# 기본 지침
system_msg = SystemMessage("You are a helpful python coding assistant.")

messages = [system_msg, HumanMessage("로또 번호 추첨 코드를 작성해줘.")]

response = model.invoke(messages)
print(response.content)

In [ ]:
# 상세한 페르소나
system_msg = SystemMessage(
    """
You are a senior Python developer with expertise in web frameworks.
Always provide code examples and explain your reasoning.
Be concise but thorough in your explanations.
"""
)

messages = [system_msg, HumanMessage("How do I create a REST API?")]

response = model.invoke(messages)
print(response.content)

### 사용자 메시지 (Human Message)

사용자 메시지는 사용자 입력 및 상호 작용을 나타냅니다. 텍스트, 이미지, 오디오, 파일 및 기타 멀티모달 콘텐츠를 포함할 수 있습니다.

In [ ]:
from langchain.messages import HumanMessage

# 메시지 객체 사용
response = model.invoke([HumanMessage("What is machine learning?")])
print(response.content)

In [ ]:
# 문자열 단축키 (HumanMessage의 단축형)
response = model.invoke("What is machine learning?")
print(response.content)

#### 메시지 메타데이터

In [ ]:
# 메타데이터 추가
human_msg = HumanMessage(
    content="Hello!",
    name="alice",  # 선택사항: 다른 사용자 식별
    id="msg_123",  # 선택사항: 추적을 위한 고유 식별자
)

human_msg

In [ ]:
response = model.invoke([human_msg])
response

### AI 메시지

AI 메시지는 모델 호출의 출력을 나타냅니다. 멀티모달 데이터, 도구 호출 및 나중에 액세스할 수 있는 제공자별 메타데이터를 포함할 수 있습니다.

In [ ]:
# 모델 호출 시 AIMessage 반환
response = model.invoke("Explain AI")
print(f"Type: {type(response)}")
print(f"Content: {response.content}")

In [ ]:
# AI 메시지를 수동으로 생성 (예: 대화 기록용)
from langchain.messages import AIMessage, SystemMessage, HumanMessage

ai_msg = AIMessage("I'd be happy to help you with that question!")

# 대화 기록에 추가
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg,  # 모델에서 온 것처럼 삽입
    HumanMessage("Great! What's 2+2?"),
]

response = model.invoke(messages)
print(response.content)

#### 도구 호출

모델이 도구 호출을 수행하면 AIMessage에 포함됩니다.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.tools import tool

model = init_chat_model("openai:gpt-4.1-mini")


@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


# 도구를 모델에 바인딩
model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Paris?")

# 도구 호출 확인
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"ID: {tool_call['id']}")

#### 토큰 사용량

AI 메시지는 `usage_metadata` 필드에 토큰 수 및 기타 사용량 메타데이터를 포함할 수 있습니다.

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4.1-mini")

response = model.invoke("Hello!")
print(response.usage_metadata)

# 출력 예시:
# {'input_tokens': 8, 'output_tokens': 304, 'total_tokens': 312, ...}

### 도구 메시지

도구 호출을 지원하는 모델의 경우 AI 메시지에 도구 호출이 포함될 수 있습니다. 도구 메시지는 단일 도구 실행 결과를 모델에 다시 전달하는 데 사용됩니다.

![](assets/tool_message.png)

In [ ]:
from langchain.messages import AIMessage, HumanMessage, ToolMessage

# 모델이 도구 호출을 수행한 후
ai_message = AIMessage(
    content=[],
    tool_calls=[
        {"name": "get_weather", "args": {"location": "서울"}, "id": "call_123"}
    ],
)
ai_message

In [ ]:
# 도구 실행 및 결과 메시지 생성
weather_result = "날씨 맑음. 섭씨 10도"
tool_message = ToolMessage(
    content=weather_result, tool_call_id="call_123"  # 호출 ID와 일치해야 함
)
tool_message

In [ ]:
# 대화 계속
messages = [
    HumanMessage("서울의 날씨는 어때?"),
    ai_message,  # 모델의 도구 호출
    tool_message,  # 도구 실행 결과
]

response = model.invoke(messages)  # 모델이 결과 처리
print(response.content)

## 메시지 콘텐츠

메시지의 콘텐츠는 모델로 전송되는 데이터의 페이로드로 생각할 수 있습니다. 메시지에는 느슨하게 타입이 지정된 `content` 속성이 있어 문자열과 타입이 지정되지 않은 객체(예: 딕셔너리) 목록을 지원합니다.

LangChain 채팅 모델은 다음을 포함할 수 있는 `content` 속성의 메시지 콘텐츠를 허용합니다:

1. 문자열
2. 제공자별 형식의 콘텐츠 블록 목록
3. LangChain의 표준 콘텐츠 블록 목록

In [ ]:
from langchain.messages import HumanMessage

# 문자열 콘텐츠
human_message = HumanMessage("Hello, how are you?")
print("String content:", human_message.content)

# 제공자별 형식 (예: OpenAI)
human_message = HumanMessage(
    content=[
        {"type": "text", "text": "Hello, how are you?"},
        {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}},
    ]
)
print("Provider-native format:", human_message.content)

# 표준 콘텐츠 블록 목록
human_message = HumanMessage(
    content_blocks=[
        {"type": "text", "text": "Hello, how are you?"},
        {"type": "image", "url": "https://example.com/image.jpg"},
    ]
)
print("Standard content blocks:", human_message.content_blocks)

### 표준 콘텐츠 블록

LangChain은 제공자 간에 작동하는 메시지 콘텐츠에 대한 표준 표현을 제공합니다.

메시지 객체는 `content` 속성을 표준적이고 타입에 안전한 표현으로 느리게 구문 분석하는 `content_blocks` 속성을 구현합니다.

In [ ]:
from langchain.messages import AIMessage

# Anthropic 형식의 메시지
message = AIMessage(
    content=[
        {
            "type": "thinking",
            "thinking": "Let me think about this...",
            "signature": "WaUjzkyp...",
        },
        {"type": "text", "text": "Here's my answer..."},
    ],
    response_metadata={"model_provider": "anthropic"},
)

# content_blocks로 표준 형식 접근
print("Standard content blocks:")
for block in message.content_blocks:
    print(block)

## 콘텐츠 블록 유형 참조

콘텐츠 블록은 타입이 지정된 딕셔너리의 목록으로 표현됩니다. 각 항목은 다음 블록 유형 중 하나를 준수해야 합니다.

### 핵심 블록 유형

**TextContentBlock** - 표준 텍스트 출력
- `type`: 항상 `"text"`
- `text`: 텍스트 콘텐츠
- `annotations`: 텍스트에 대한 주석 목록 (선택)
- `extras`: 추가 제공자별 데이터 (선택)

**ReasoningContentBlock** - 모델 추론 단계
- `type`: 항상 `"reasoning"`
- `reasoning`: 추론 콘텐츠
- `extras`: 추가 제공자별 데이터 (선택)

In [ ]:
# 텍스트 블록 예시
text_block = {"type": "text", "text": "Hello world", "annotations": []}

# 추론 블록 예시
reasoning_block = {
    "type": "reasoning",
    "reasoning": "The user is asking about...",
    "extras": {"signature": "abc123"},
}